In [13]:
import os
import torch
import requests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig


In [14]:
def config():
    load_dotenv()

config()

In [15]:
API_URL = "https://api-inference.huggingface.co/models/google/gemma-7b-it"
headers = {"Authorization": "Bearer {}".format(os.getenv('API_KEY'))}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)    
    return response.json()


In [16]:
def output(prompt):
    text =  "role: User" + "\ncontent: {}".format(prompt)   
    
    return query({
        "inputs":text,
        "parameters":{"top_k":2,
                     "top_p":0.9,
                     "temperature":0.2,
                     "max_length":100}  
        
    })[0]['generated_text'][len(text):].strip()



In [17]:
print(output("what is capital of india?"))

**Answer:** Delhi.

The capital of India is Delhi.
